<h1> Лабораторная работа 6 </h1>
Загружаем библиотеки

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier

Подготавливаем данные для обучения: балансируем количество легальных и мошеннических транзакци, определяем таргет и оставляем поля, которые имеют важную информацию для классификации информацию

In [ ]:
df = pd.read_csv("fraud.csv",sep=',')

features = ['trans_date_trans_time', 'amt', 'category', 'merchant', 'city', 'state', 'job', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop', 'cc_num', 'gender']
target = 'is_fraud'

fraud=df[df[target]==1]
not_fraud=df[df[target]==0]
not_fraud_down_sample=not_fraud.sample(n=20000)
under=pd.concat([fraud,not_fraud_down_sample])

x = under[features]
y = under[target]

x['hour']=pd.to_datetime(x['trans_date_trans_time'],dayfirst=True).dt.hour
x=x.drop(['trans_date_trans_time'],axis=1)
categorical_features = ['category', 'merchant', 'city', 'state', 'job', 'gender']
numeric_features = ['amt', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop', 'cc_num', 'hour']

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
x = preprocessor.fit_transform(x)

Разделяем данные на обучающиеся и тестируемые


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.2)

Определим наилучшие гиперпараметры: количество деревьев, скорость обучения и глубину дерева

In [7]:
cat_boost_classifier = CatBoostClassifier()
cat_boost_params ={
    "iterations": range(1,6),
    'learning_rate': [0.1,0.25,0.3,0.5,0.75,1],
    'depth':range(1,6),
}
cat_boost_grid = GridSearchCV(cat_boost_classifier, cat_boost_params, cv=2, n_jobs=1,verbose=3)
cat_boost_grid.fit(x_train, y_train)
best_cat_boost_params = cat_boost_grid.best_params_

Fitting 2 folds for each of 125 candidates, totalling 250 fits
0:	learn: 0.5658017	total: 48.3ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.1;, score=0.944 total time=   0.1s
0:	learn: 0.5563920	total: 1.51ms	remaining: 0us
[CV 2/2] END depth=1, iterations=1, learning_rate=0.1;, score=0.950 total time=   0.1s
0:	learn: 0.4185051	total: 1.65ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.25;, score=0.944 total time=   0.1s
0:	learn: 0.4007738	total: 1.59ms	remaining: 0us
[CV 2/2] END depth=1, iterations=1, learning_rate=0.25;, score=0.950 total time=   0.1s
0:	learn: 0.2710922	total: 1.57ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.5;, score=0.944 total time=   0.1s
0:	learn: 0.2499873	total: 1.51ms	remaining: 0us
[CV 2/2] END depth=1, iterations=1, learning_rate=0.5;, score=0.950 total time=   0.1s
0:	learn: 0.2087313	total: 1.57ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.75;, score=0.944 t

Используем наилучшие параметры, найденные с помощью перекрестной проверки, и оценим качество модели.

In [8]:
cat_best = CatBoostClassifier(**best_cat_boost_params)
cat_best.fit(x_train,y_train)
y_pred=cat_best.predict(x_test)
print(metrics.classification_report(y_test,y_pred,digits=5))

0:	learn: 0.1965294	total: 5.66ms	remaining: 22.6ms
1:	learn: 0.1371554	total: 11.9ms	remaining: 17.9ms
2:	learn: 0.1303626	total: 17.4ms	remaining: 11.6ms
3:	learn: 0.0984285	total: 22.9ms	remaining: 5.72ms
4:	learn: 0.0849484	total: 29.8ms	remaining: 0us
              precision    recall  f1-score   support

           0    0.97692   0.99299   0.98489     15988
           1    0.92355   0.78299   0.84748      1728

    accuracy                        0.97251     17716
   macro avg    0.95024   0.88799   0.91619     17716
weighted avg    0.97172   0.97251   0.97149     17716

